# Simple Recommender System

In this project, I create a simple movie scoring algorithm that predicts a users movie ratings based on that user's existing movie ratings and the ratings of similar users. The data is from the MovieLens database (details in the Data section). I use proximal gradient descent as the optimization algorithm for the penalized problem. 

Here, we are solving the objective function:

$$
\mbox{minimize} \ \lambda \|X\|_* + \sum_{ij \ | \ A_{ij} \ is \ known} (X_{ij} - A_{ij})^2,
$$
where $\|X\|_*$ is the nuclear norm and matrix $A$ is the data matrix provided (users and real ratings). This algorithm requires thresholding the singular values of $X$ at each iteration. Note that this is a convex relaxation of the original recommender system formulation, which attempts to minimize the rank of the matrix X (NP-Hard).

The idea here is that we have a table of movies and many viewers, and each viewer as watch some tiny fraction of the movies and rated them. The output of this algorithm is the completed table, where the missing ratings for each movie are inferred from group data (similar users). 

# Data
Datasets: http://files.grouplens.org/datasets/movielens/ml-100k/

There are two files in this repository from the MovieLens database that I use. One is the base data (matrix $A$), and the other is actual ratings for testing purposes

1. u1.base
2. u1.test

The full u data set is 100000 ratings by 943 users on 1682 items. Each user has rated at least 20 movies. Users and items are numbered consecutively from 1. The data is randomly ordered. 

The u1.base and u1.test files are a subset of the full u dataset, and are tab separated lists of:   
user id | item id | rating | timestamp

* The timestamp feature in the dataset has been ignored (not needed for this problem).
* Additional information on the datasets available here: http://dx.doi.org/10.1145/2827872

# Scoring
Let $X$ be the output of the proximal gradient method, let $A'$ and $\Omega'$ be the data matrix and the non-zero entry list for the test data. Then the score is 
$$
\frac{1}{|\Omega'|} \sum_{(i,j)\in\Omega'} | X_{ij} - A_{ij}'|.
$$
In other words, the average absolute deviation of the computed ratings from the predicted ratings 

   

# Implement Recommender

In [28]:
import scipy.sparse
import scipy.linalg
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt

In [8]:
# Function to load data into desired form
def read_ml(fname, sep = '\t'):
    ridx, cidx, d = [], [], []
    with open(fname) as f:
        for l in f:
            r, c, v, _ = l.split(sep)
            ridx.append(int(r))
            cidx.append(int(c))
            d.append(int(v))
    A = scipy.sparse.coo_matrix((d,(ridx,cidx))).tocsr()
    W = scipy.sparse.coo_matrix(([1]*len(ridx),(ridx, cidx))).tocsr()
    return A, W

# Functions for Recommender
def singular_value_thresholding(X, threshold):
    U, s, Vh = sp.linalg.svd(X, full_matrices=False, compute_uv=True, overwrite_a=True)
    return U@sp.sparse.diags(np.maximum(s - threshold, 0))@Vh

def converged(X, prox_X, epsilon): # Check for convergence
    return np.linalg.norm(X - prox_X, 'fro') < epsilon
    
def nuclear_norm_minimization(A, W, lambda_, maxIter=100, epsilon=1.0e-1): # Minimize nuclear norm
    X = np.random.uniform(1, 5, size=A.shape)
    X[A.nonzero()] = A.data
    for _ in range(maxIter):
        dX = (W.multiply(X)-A).multiply(2).toarray()
        prev_X, X = X, singular_value_thresholding(X - dX, lambda_)
        if converged(prev_X, X, epsilon): break
    return X

def report_score(X, At): #Function to calculate score
    score = sum(np.abs(X[At.nonzero()]-At.data))/At.count_nonzero()
    print(f'Average absolute deviation: {score:.6f}')
    def objFunc(self, X, A, s):
        g_x = self.gx(A, s)
        f_x = self.fx(X, A)
        return g_x + f_x

    def gradient(self, X, A): # Define objective function gradient wrt x
        x_gradient = np.zeros_like(A)
        x_gradient[knownValues] = 2 * (X[knownValues] - A[knownValues])
        return x_gradient

In [9]:
lambda_ = 10 # selected via tuning
A, W = read_ml('u1.base')
X = nuclear_norm_minimization(A, W, lambda_, maxIter=200)
report_score(X, read_ml('u1.test')[0])

Average absolute deviation: 0.755794


# Summary

Here we can see that the recommender correctly calculated user movie scores to within 0.75 rating points (on average) of the actual values given in the test data!